In [5]:
import pandas as pd
import numpy as np
import re
import  matplotlib.pyplot  as plt
% matplotlib inline
% load_ext autoreload
% autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
lifeExpPath = 'data/IHME_USA_COUNTY_LE_ONLY_1980_2014_NATIONAL_Y2017M05D08.csv'
censusPath = 'data/cc-est2016-alldata.csv'
leCols = ['Location', 'FIPS', 'Life expectancy, 2010*', 'Life expectancy, 2014*']
lifeExpDF = pd.read_csv(lifeExpPath, dtype = {'FIPS': int}, skiprows = [1, 3195, 3196],
                       usecols = leCols)
censusCols = ["STATE", "COUNTY", "YEAR", "AGEGRP", "TOT_POP", "STNAME", "CTYNAME",# general information.
              "IA_FEMALE", "IA_MALE", "IAC_FEMALE", "IAC_MALE",# American Indian and Alaskan Native only
                                                               # or American Indian and Alaskan Native only+Combo
              "NA_FEMALE", "NA_MALE", "NAC_FEMALE", "NAC_MALE",]# Same as above but with Native Hawaiians and
                                                               # pacific islanders
#               "HIA_FEMALE", "HIA_MALE", "HIAC_FEMALE", "HIAC_MALE", # Same as the first group but with Hispanics
#               "HNA_FEMALE", "HNA_MALE", "HNAC_FEMALE", "HNAC_MALE"] # Same as second group but with Hispanics
censusDF = pd.read_csv(censusPath, encoding = "ISO-8859-1", usecols = censusCols,
                      dtype = {"STATE": str, "COUNTY": str})
censusDF = censusDF[censusDF.AGEGRP == 0]
censusDF = censusDF[censusDF.YEAR.isin([3, 7])]# these years correspond to 2010 and 2014. 2010 is a bit weird
# in that there are 3 estimates for it. As far as I can tell the third one is the most like the 2014 census data.
# this was determined using the census Data methedology pdf.
censusDF.STATE = censusDF.STATE.apply(lambda x: x.lstrip("0"))
censusDF.insert(0, "FIPS", censusDF.STATE + censusDF.COUNTY)
censusDF.drop(["STATE", "COUNTY"], axis = 1, inplace = True)
censusDF.FIPS = censusDF.FIPS.astype(int)

In [7]:
# combine male and female populations
def combineMaleAndFemalePopulations(df, prefix):
    colsToCombine = [idx for idx, col in enumerate(list(df)) if re.match(pattern = prefix, string = col) is not None]
    print("Combining Columns: {} at indices {}".format(np.array(list(df))[colsToCombine], colsToCombine))
    combinedCol = df.iloc[:,colsToCombine].sum(axis = 1)
    return(combinedCol)

In [8]:
censusDF.insert(8, "IA_TOTAL", combineMaleAndFemalePopulations(df = censusDF, prefix = "^IA_.*"))
censusDF.insert(11, "NA_TOTAL", combineMaleAndFemalePopulations(df = censusDF, prefix = "^NA_.*"))
censusDF.insert(14, "IAC_TOTAL", combineMaleAndFemalePopulations(df = censusDF, prefix = "^IAC_.*"))
censusDF.insert(17, "NAC_TOTAL", combineMaleAndFemalePopulations(df = censusDF, prefix = "^NAC_.*"))

Combining Columns: ['IA_MALE' 'IA_FEMALE'] at indices [6, 7]
Combining Columns: ['NA_MALE' 'NA_FEMALE'] at indices [9, 10]
Combining Columns: ['IAC_MALE' 'IAC_FEMALE'] at indices [12, 13]
Combining Columns: ['NAC_MALE' 'NAC_FEMALE'] at indices [15, 16]


In [9]:
censusDF

,FIPS,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,IA_MALE,IA_FEMALE,IA_TOTAL,NA_MALE,NA_FEMALE,NA_TOTAL,IAC_MALE,IAC_FEMALE,IAC_TOTAL,NAC_MALE,NAC_FEMALE,NAC_TOTAL
38,1001,Alabama,Autauga County,3,0,54742,116,135,251,29,17,46,258,301,559,42,33,75
114,1001,Alabama,Autauga County,7,0,54977,122,153,275,32,20,52,286,307,593,52,47,99
209,1003,Alabama,Baldwin County,3,0,183199,664,674,1338,76,54,130,1303,1333,2636,124,117,241
285,1003,Alabama,Baldwin County,7,0,199745,759,756,1515,63,54,117,1506,1551,3057,137,145,282
380,1005,Alabama,Barbour County,3,0,27348,87,72,159,29,17,46,150,108,258,35,19,54
456,1005,Alabama,Barbour County,7,0,26763,91,66,157,26,17,43,150,101,251,31,22,53
551,1007,Alabama,Bibb County,3,0,22861,35,33,68,27,4,31,83,79,162,31,7,38
627,1007,Alabama,Bibb County,7,0,22511,43,41,84,24,3,27,78,91,169,30,10,40
722,1009,Alabama,Blount County,3,0,57376,179,176,355,36,24,60,356,376,732,47,37,84
798,1009,Alabama,Blount County,7,0,57621,187,167,354,39,22,61,367,413,780,51,40,91


In [10]:
# extract out the census data into year specific dataframes
censusDF_2010 = censusDF[censusDF.YEAR == 3]
censusDF_2014 = censusDF[censusDF.YEAR == 7]

In [11]:
# Change from raw population to percentage of total of county population.
censusDF_2014.iloc[:,6:] = censusDF_2014.iloc[:,6:].divide(censusDF_2014.TOT_POP, axis = 'index')
censusDF_2010.iloc[:,6:] = censusDF_2010.iloc[:,6:].divide(censusDF_2010.TOT_POP, axis = 'index')

/home/ob2285/anaconda3/envs/lifeExpCount/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
# Remove confidence intervals and convert LE to a float
lifeExpDF['Life expectancy, 2010*'] = lifeExpDF['Life expectancy, 2010*'].apply(lambda x: float(x.split()[0]))
lifeExpDF['Life expectancy, 2014*'] = lifeExpDF['Life expectancy, 2014*'].apply(lambda x: float(x.split()[0]))

In [13]:
lifeExpDF.head()

,Location,FIPS,"Life expectancy, 2010*","Life expectancy, 2014*"
0,Alabama,1,75.58,75.65
1,"Autauga County, Alabama",1001,75.74,75.67
2,"Baldwin County, Alabama",1003,77.80,78.08
3,"Barbour County, Alabama",1005,75.34,75.42
4,"Bibb County, Alabama",1007,74.13,73.97


In [14]:
censusDF.head()

,FIPS,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,IA_MALE,IA_FEMALE,IA_TOTAL,NA_MALE,NA_FEMALE,NA_TOTAL,IAC_MALE,IAC_FEMALE,IAC_TOTAL,NAC_MALE,NAC_FEMALE,NAC_TOTAL
38,1001,Alabama,Autauga County,3,0,54742,116,135,251,29,17,46,258,301,559,42,33,75
114,1001,Alabama,Autauga County,7,0,54977,122,153,275,32,20,52,286,307,593,52,47,99
209,1003,Alabama,Baldwin County,3,0,183199,664,674,1338,76,54,130,1303,1333,2636,124,117,241
285,1003,Alabama,Baldwin County,7,0,199745,759,756,1515,63,54,117,1506,1551,3057,137,145,282
380,1005,Alabama,Barbour County,3,0,27348,87,72,159,29,17,46,150,108,258,35,19,54


In [15]:
censusDF_2010.head()

,FIPS,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,IA_MALE,IA_FEMALE,IA_TOTAL,NA_MALE,NA_FEMALE,NA_TOTAL,IAC_MALE,IAC_FEMALE,IAC_TOTAL,NAC_MALE,NAC_FEMALE,NAC_TOTAL
38,1001,Alabama,Autauga County,3,0,54742,0.002119,0.002466,0.004585,0.000530,0.000311,0.000840,0.004713,0.005499,0.010212,0.000767,0.000603,0.001370
209,1003,Alabama,Baldwin County,3,0,183199,0.003624,0.003679,0.007304,0.000415,0.000295,0.000710,0.007112,0.007276,0.014389,0.000677,0.000639,0.001316
380,1005,Alabama,Barbour County,3,0,27348,0.003181,0.002633,0.005814,0.001060,0.000622,0.001682,0.005485,0.003949,0.009434,0.001280,0.000695,0.001975
551,1007,Alabama,Bibb County,3,0,22861,0.001531,0.001444,0.002974,0.001181,0.000175,0.001356,0.003631,0.003456,0.007086,0.001356,0.000306,0.001662
722,1009,Alabama,Blount County,3,0,57376,0.003120,0.003067,0.006187,0.000627,0.000418,0.001046,0.006205,0.006553,0.012758,0.000819,0.000645,0.001464


In [17]:
censusDF_2014.head()

,FIPS,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,IA_MALE,IA_FEMALE,IA_TOTAL,NA_MALE,NA_FEMALE,NA_TOTAL,IAC_MALE,IAC_FEMALE,IAC_TOTAL,NAC_MALE,NAC_FEMALE,NAC_TOTAL
114,1001,Alabama,Autauga County,7,0,54977,0.002219,0.002783,0.005002,0.000582,0.000364,0.000946,0.005202,0.005584,0.010786,0.000946,0.000855,0.001801
285,1003,Alabama,Baldwin County,7,0,199745,0.003800,0.003785,0.007585,0.000315,0.000270,0.000586,0.007540,0.007765,0.015305,0.000686,0.000726,0.001412
456,1005,Alabama,Barbour County,7,0,26763,0.003400,0.002466,0.005866,0.000971,0.000635,0.001607,0.005605,0.003774,0.009379,0.001158,0.000822,0.001980
627,1007,Alabama,Bibb County,7,0,22511,0.001910,0.001821,0.003732,0.001066,0.000133,0.001199,0.003465,0.004042,0.007507,0.001333,0.000444,0.001777
798,1009,Alabama,Blount County,7,0,57621,0.003245,0.002898,0.006144,0.000677,0.000382,0.001059,0.006369,0.007168,0.013537,0.000885,0.000694,0.001579


In [18]:
# merge dataframes on FIPS and remove some redundant columns
lifeExpRaceDF_2010 = lifeExpDF.merge(censusDF_2010, how = "inner", on = "FIPS")
lifeExpRaceDF_2014 = lifeExpDF.merge(censusDF_2014, how = "inner", on = "FIPS")
lifeExpRaceDF_2010.drop(labels = ["YEAR", "AGEGRP"], axis = 1, inplace = True)
lifeExpRaceDF_2014.drop(labels = ["YEAR", "AGEGRP"], axis = 1, inplace = True)

In [19]:
lifeExpRaceDF_2010.head(n = 5)

,Location,FIPS,"Life expectancy, 2010*","Life expectancy, 2014*",STNAME,CTYNAME,TOT_POP,IA_MALE,IA_FEMALE,IA_TOTAL,NA_MALE,NA_FEMALE,NA_TOTAL,IAC_MALE,IAC_FEMALE,IAC_TOTAL,NAC_MALE,NAC_FEMALE,NAC_TOTAL
0,"Autauga County, Alabama",1001,75.74,75.67,Alabama,Autauga County,54742,0.002119,0.002466,0.004585,0.000530,0.000311,0.000840,0.004713,0.005499,0.010212,0.000767,0.000603,0.001370
1,"Baldwin County, Alabama",1003,77.80,78.08,Alabama,Baldwin County,183199,0.003624,0.003679,0.007304,0.000415,0.000295,0.000710,0.007112,0.007276,0.014389,0.000677,0.000639,0.001316
2,"Barbour County, Alabama",1005,75.34,75.42,Alabama,Barbour County,27348,0.003181,0.002633,0.005814,0.001060,0.000622,0.001682,0.005485,0.003949,0.009434,0.001280,0.000695,0.001975
3,"Bibb County, Alabama",1007,74.13,73.97,Alabama,Bibb County,22861,0.001531,0.001444,0.002974,0.001181,0.000175,0.001356,0.003631,0.003456,0.007086,0.001356,0.000306,0.001662
4,"Blount County, Alabama",1009,76.41,76.16,Alabama,Blount County,57376,0.003120,0.003067,0.006187,0.000627,0.000418,0.001046,0.006205,0.006553,0.012758,0.000819,0.000645,0.001464


In [20]:
lifeExpRaceDF_2014.head(n = 5)

,Location,FIPS,"Life expectancy, 2010*","Life expectancy, 2014*",STNAME,CTYNAME,TOT_POP,IA_MALE,IA_FEMALE,IA_TOTAL,NA_MALE,NA_FEMALE,NA_TOTAL,IAC_MALE,IAC_FEMALE,IAC_TOTAL,NAC_MALE,NAC_FEMALE,NAC_TOTAL
0,"Autauga County, Alabama",1001,75.74,75.67,Alabama,Autauga County,54977,0.002219,0.002783,0.005002,0.000582,0.000364,0.000946,0.005202,0.005584,0.010786,0.000946,0.000855,0.001801
1,"Baldwin County, Alabama",1003,77.80,78.08,Alabama,Baldwin County,199745,0.003800,0.003785,0.007585,0.000315,0.000270,0.000586,0.007540,0.007765,0.015305,0.000686,0.000726,0.001412
2,"Barbour County, Alabama",1005,75.34,75.42,Alabama,Barbour County,26763,0.003400,0.002466,0.005866,0.000971,0.000635,0.001607,0.005605,0.003774,0.009379,0.001158,0.000822,0.001980
3,"Bibb County, Alabama",1007,74.13,73.97,Alabama,Bibb County,22511,0.001910,0.001821,0.003732,0.001066,0.000133,0.001199,0.003465,0.004042,0.007507,0.001333,0.000444,0.001777
4,"Blount County, Alabama",1009,76.41,76.16,Alabama,Blount County,57621,0.003245,0.002898,0.006144,0.000677,0.000382,0.001059,0.006369,0.007168,0.013537,0.000885,0.000694,0.001579


In [21]:
# Write those puppies to csv my dude
lifeExpRaceDF_2010.to_csv("data/LE_censusData_2010.csv")
lifeExpRaceDF_2010.to_csv("data/LE_censusData_2014.csv")

In [22]:
# this information is also in the censusDataKey pdf but it's helpful to put it here too
yearDict = {"1":"4/1/2010 Census population",
            "2":"4/1/2010 population estimates base",
            "3":"7/1/2010 population estimate",
            "4":"7/1/2011 population estimate",
            "5":"7/1/2012 population estimate",
            "6":"7/1/2013 population estimate",
            "7":"7/1/2014 population estimate",
            "8":"7/1/2015 population estimate",
            "9":"7/1/2016 population estimate",}# Maps year ID to human readable year
ageGrpDict = {"0":"Total",
              "1":"Age 0 to 4 years",
              "2":"Age 5 to 9 years",
              "3":"Age 10 to 14 years",
              "4":"Age 15 to 19 years",
              "5":"Age 20 to 24 years",
              "6":"Age 25 to 29 years",
              "7":"Age 30 to 34 years",
              "8":"Age 35 to 39 years",
              "9":"Age 40 to 44 years",
              "10":"Age 45 to 49 years",
              "11":"Age 50 to 54 years",
              "12":"Age 55 to 59 years",
              "13":"Age 60 to 64 years",
              "14":"Age 65 to 69 years",
              "15":"Age 70 to 74 years",
              "16":"Age 75 to 79 years",
              "17":"Age 80 to 84 years",
              "18":"Age 85 years or older",}# Maps age group ID to year. 